In [7]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from textblob import TextBlob
from scipy.sparse import csr_matrix
from nltk.corpus import words
from tqdm import tqdm
english_corpus = set(words.words())

matrix = pd.read_csv('matrix.csv')
#matrix = matrix.iloc[:, :100]
#pd.DataFrame(list(matrix.columns), columns = ['feature']).to_csv('matrix_columns.csv', index = False)
matrix.head()

,aa,aaa,aaaaaaaa,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa,aaaaand,aaas,aab,aabad,aabout,aaca,...,zurich,zvun,zwaar,zwkmjkkz,zxcap,zxje,zxlo,zy,zz,zzz
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
import nltk

from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('words')
from nltk.corpus import words

words.words()


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/mrugeles/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to /home/mrugeles/nltk_data...
[nltk_data]   Package words is already up-to-date!


['A',
 'a',
 'aa',
 'aal',
 'aalii',
 'aam',
 'Aani',
 'aardvark',
 'aardwolf',
 'Aaron',
 'Aaronic',
 'Aaronical',
 'Aaronite',
 'Aaronitic',
 'Aaru',
 'Ab',
 'aba',
 'Ababdeh',
 'Ababua',
 'abac',
 'abaca',
 'abacate',
 'abacay',
 'abacinate',
 'abacination',
 'abaciscus',
 'abacist',
 'aback',
 'abactinal',
 'abactinally',
 'abaction',
 'abactor',
 'abaculus',
 'abacus',
 'Abadite',
 'abaff',
 'abaft',
 'abaisance',
 'abaiser',
 'abaissed',
 'abalienate',
 'abalienation',
 'abalone',
 'Abama',
 'abampere',
 'abandon',
 'abandonable',
 'abandoned',
 'abandonedly',
 'abandonee',
 'abandoner',
 'abandonment',
 'Abanic',
 'Abantes',
 'abaptiston',
 'Abarambo',
 'Abaris',
 'abarthrosis',
 'abarticular',
 'abarticulation',
 'abas',
 'abase',
 'abased',
 'abasedly',
 'abasedness',
 'abasement',
 'abaser',
 'Abasgi',
 'abash',
 'abashed',
 'abashedly',
 'abashedness',
 'abashless',
 'abashlessly',
 'abashment',
 'abasia',
 'abasic',
 'abask',
 'Abassin',
 'abastardize',
 'abatable',
 'abate

In [5]:

from tqdm import tqdm
import numpy as np

from textblob import Word

def spellcheck(word, threshold):
    value = Word(word).spellcheck()[0]
    if (value[1] < threshold):
        return f'-1'
    return value[0]


def correct(text):
    return TextBlob(text).correct().string

np_spellcheck = np.vectorize(spellcheck)

def process_words(columns):
    word_list = [(word, spellcheck(word, 0.7)) for word in columns]
    word_list = dict(word_list)
    return word_list
    


SyntaxError: invalid syntax (<ipython-input-5-7ba134dfef35>, line 1)

In [ ]:
columns_df = pd.DataFrame(list(matrix.columns), columns = ['feature'])
columns_df

In [ ]:
tqdm.pandas(desc="feature_spellcheck")
%time columns_df['feature_spellcheck'] = columns_df['feature'].progress_apply(lambda word: spellcheck(word, 0.7))
columns_df

In [ ]:
print(matrix.shape)
matrix.head()

In [ ]:
matrix = matrix.drop(columns_df.loc[columns_df['feature_spellcheck'] == "-1"]['feature'].values, axis = 1)

print(matrix.shape)
matrix.head()

In [ ]:
renamed_columns = dict(columns_df.loc[columns_df['feature_spellcheck'] != "-1"].to_dict('split')['data'])


In [ ]:
matrix = matrix.rename(columns = renamed_columns)
matrix.head()

In [ ]:
for column in tqdm(matrix.columns[matrix.columns.duplicated()]):
    matrix[f'{column}_duplicated'] = matrix[column].groupby(level = 0, axis = 1).sum()

matrix = matrix.drop(matrix.columns[matrix.columns.duplicated()], axis = 1)
print(matrix.shape)
matrix.head()

In [ ]:
matrix.rename(columns=lambda x: x.replace('_duplicated', ''), inplace=True)
print(matrix.shape)
matrix.head()

In [ ]:
matrix = matrix.applymap(lambda n: 1 if n > 1 else n)

In [ ]:
from textblob import Word
word = Word('abdominal')


In [ ]:
word.lemmatize(pos='v')

In [ ]:
word.stem()

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

nltk.download('stopwords')

def tokenize(text):
    text = text.lower()
    text = re.sub(r"[^a-z]", " ", text)
    word_list = word_tokenize(text)
    word_list = [w for w in word_list if w not in stopwords.words("english")]
    word_list = [WordNetLemmatizer().lemmatize(w, pos='v') for w in word_list]
    return word_list

%time tokenize('I am in Thomassin number 32, in the area named Pyron. I would like to have some water. Thank God we are fine, but we desperately need water. Thanks')

In [ ]:
def tokenize_blob(text):
    text = text.lower()
    text = re.sub(r"[^a-z]", " ", text)
    text_blob = TextBlob(text)
    word_list = [w for w in text_blob.words if w not in stopwords.words("english")]
    word_list = [word.lemmatize(pos='v') for word in word_list]
    return word_list
    
%time tokenize_blob('I am in Thomassin number 32, in the area named Pyron. I would like to have some water. Thank God we are fine, but we desperately need water. Thanks')

In [ ]:
s1 = pd.Series([0, 0, 1, 0, 1, 1, 0])
s1

In [ ]:
s2 = pd.Series([0, 1, 0, 0, 1, 0, 0])
s2

In [ ]:
s3 = pd.Series([0, 1, 0, 0, 1, 0, 0])
s3

In [ ]:
s1.combine(s2, max)

In [ ]:
df.iloc[:, 0]

In [ ]:
df = pd.concat([s1, s2, s3], axis = 1)
display(df)

indexes = [0, 1, 2]
for idx in indexes[1:]:
    df.iloc[:, indexes[0]] = df.iloc[:, indexes[0]].combine(df.iloc[:, idx], max)


df = df.drop(df.columns[indexes[1:]], axis = 1)
df

In [3]:
f = open("matrix_renamed.csv", "r")
columns = f.readline().replace('\n', '').split(',')

matrix = pd.read_csv('matrix_renamed.csv')
matrix.columns = columns
matrix.head()

,cab,bad,abandon,abate,about,caleche,has,ability,abject,able,...,maman,labia,zealand,zero,change,hung,i,zinc,zone,good
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
matrix.iloc[:, [468]]

In [ ]:
matrix.shape

In [ ]:
matrix.columns = columns
matrix.head()

In [ ]:
np.where(matrix.columns.values == 'bamboo')[0]

In [ ]:
matrix.iloc[:, 468]

In [ ]:
matrix.shape

In [ ]:
df = matrix.drop(matrix.columns[469], axis = 1)
df.shape

In [ ]:
matrix.iloc[:, [468, 469, 470, 473]]

In [ ]:
matrix.columns[matrix.columns.duplicated()]

In [ ]:
np.where(matrix.columns.values == 'advice')[0]

In [ ]:
import numpy as np
print(matrix.columns.values)
column_indexes = np.where(matrix.columns.values == 'advice')[0]
print(column_indexes)
for idx in column_indexes:
    print(idx)
#column_indexes

In [14]:


def drop_duplicated(matrix):
    for column in tqdm(matrix.columns[matrix.columns.duplicated()]):
        matrix[f'{column}_duplicated'] = matrix[column].groupby(level = 0, axis = 1).sum()
        matrix[f'{column}_duplicated'] = matrix[f'{column}_duplicated'].apply(lambda n: 1 if n > 1 else n)
    
    matrix = matrix.drop(matrix.columns[matrix.columns.duplicated()], axis = 1)
    matrix.rename(columns=lambda x: x.replace('_duplicated', ''), inplace=True)
    return matrix

%time matrix1 = drop_duplicated(matrix.copy())

100%|██████████| 802/802 [02:48<00:00,  4.75it/s]


CPU times: user 6min 2s, sys: 5min 8s, total: 11min 11s
Wall time: 2min 49s


/home/mrugeles/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4223: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)


In [15]:
matrix1.shape

(2622, 5252)

In [ ]:
matrix1['bamboo']

In [11]:
def drop_duplicated_loc(matrix):
    processed_columns = []    

    for column in tqdm(matrix.columns[matrix.columns.duplicated()]):
        if(column not in processed_columns):
            column_indexes = np.where(matrix.columns.values == column)[0]
            
            for idx in column_indexes[1:]:
                matrix.iloc[:, column_indexes[0]] = matrix.iloc[:, column_indexes[0]].combine(matrix.iloc[:, idx], max)
            processed_columns += [column]
    matrix = matrix.iloc[:,~matrix.columns.duplicated()]
    return matrix

%time matrix2 = drop_duplicated_loc(matrix.copy())

100%|██████████| 802/802 [00:40<00:00, 19.61it/s]

CPU times: user 41.5 s, sys: 406 ms, total: 41.9 s
Wall time: 41.1 s


In [12]:
matrix2

,cab,bad,abandon,abate,about,caleche,has,ability,abject,able,...,couture,y,juan,duncan,z,maman,zealand,zero,zinc,zone
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2617,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2618,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2619,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2620,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
matrix2['bamboo']

0       0
1       0
2       0
3       0
4       0
       ..
2617    0
2618    0
2619    0
2620    0
2621    0
Name: bamboo, Length: 2622, dtype: int64

In [3]:
model_features = pd.read_csv('model_features.csv')
model_features.loc[model_features['feature'].isna()]

,feature
6969,NaN
7198,NaN


In [3]:
model_features.loc[6965:6975]

,feature
6965,myself
6966,mysterious
6967,n
6968,na
6969,NaN
6970,navy
6971,nasi
6972,radial
6973,pagan
6974,agar


In [ ]:
model_features.loc[7196:72]